# Исследование надежности заемщиков.

В этой работе мы очистим данные и проведем исследование факторов которые влияют на возврат кредита заемщиком.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Обнаружил два столбца с пропущенными данными - days_employed и total_income. В обоих столбцах равное количество значений - похоже есть какая то связь.

In [4]:
data.isna().mean()*100

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

Доля пропусков >10%, нельзя их просто удалить - надо разбираться почему они появились.

### Шаг 2.1 Заполнение пропусков

In [5]:
print(data['days_employed'].isna().sum())

2174


In [6]:
data['days_employed'].isna().mean()*100

10.099883855981417

In [7]:
print(data['total_income'].isna().sum())

2174


In [8]:
data['total_income'].isna().mean()*100

10.099883855981417

In [9]:
print(data['total_income'].min())
print(data['total_income'].max())
print(data['total_income'].mean())
print(data['total_income'].median())
print(data['total_income'].mean() - data['total_income'].median())

20667.26379327158
2265604.028722744
167422.30220817294
145017.93753253992
22404.364675633027


Мы нашли два столбца с пропущенными значениями - total_income и days_employed. В обоих столбцах пропущенные значения совпадают по строкам, количеству и доле. Скорее всего, люди просто не предоставили информации о своем месте работы, поэтому мы не видим ни зарплаты ни стажа. В столбце days_employed значения стали отрицательными, наверняка это техническая ошибка. Мы видим что минимальное и максимальное значение в столбце total_income сильно различаются, поэтому медиана среднего дохода будет более репрезентативна чем среднее арифметическое(а по хорошему надо еще смотреть на дисперсию и нормальное распределение:)). Заполним медианой пропущенные значения.

In [10]:
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
print(data['total_income'].isna().sum())

0


### Шаг 2.2 Проверка данных на аномалии и исправления.

In [11]:
print(data['days_employed'])
print(data['days_employed'].isna().sum())

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64
2174


In [12]:
data.value_counts('children')

children
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
dtype: int64

В столбце children есть значение -1. Невозможно понять это опечатка или какая то техническая ошибка, проще удалить эти строки - их всего 47.

In [13]:
data = data[data['children'] != -1]
data.reset_index()

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21473,21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21474,21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21475,21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21476,21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


20 детей это тоже странно, посмотрим на этих клиентов.

In [14]:
data.query('children == 20')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,145017.937533,покупка жилой недвижимости
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


20 - это тоже не реалистичное число, особенно учитывая отсутствие значений 6-19.

In [15]:
data = data[data['children'] != 20]
data.reset_index()

,index,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21397,21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21398,21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21399,21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21400,21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [16]:
data['days_employed'] = data['days_employed'].abs()
print(data['days_employed'].min())
print(data['days_employed'].max())

24.14163324048118
401755.40047533


Разброс стажа тоже достаточно большой, поэтому и в этом случае мы заполним пропуски медианой.

In [17]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
print(data['days_employed'].isna().sum())

0


In [18]:
data['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

 Гендер XNA? Будем считать что это nonbinary gender -_-

### Шаг 2.3. Изменение типов данных.

В целях оптимизации данных изменим тип данных days_employed и total_income на int.

In [19]:
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21402 non-null  int64 
 1   days_employed     21402 non-null  int64 
 2   dob_years         21402 non-null  int64 
 3   education         21402 non-null  object
 4   education_id      21402 non-null  int64 
 5   family_status     21402 non-null  object
 6   family_status_id  21402 non-null  int64 
 7   gender            21402 non-null  object
 8   income_type       21402 non-null  object
 9   debt              21402 non-null  int64 
 10  total_income      21402 non-null  int64 
 11  purpose           21402 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


In [20]:
data[data.select_dtypes(np.float64).columns] = data.select_dtypes(np.float64).astype(np.float32)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21402 non-null  int64 
 1   days_employed     21402 non-null  int64 
 2   dob_years         21402 non-null  int64 
 3   education         21402 non-null  object
 4   education_id      21402 non-null  int64 
 5   family_status     21402 non-null  object
 6   family_status_id  21402 non-null  int64 
 7   gender            21402 non-null  object
 8   income_type       21402 non-null  object
 9   debt              21402 non-null  int64 
 10  total_income      21402 non-null  int64 
 11  purpose           21402 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Шаг 2.4. Удаление дубликатов.

In [21]:
data = data.drop_duplicates().reset_index(drop=True)

Приведем все значения к нижнему регистру.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [30]:
education = data[['education_id', 'education']]

,education_id,education
0,0,высшее
1,1,среднее
2,1,Среднее
3,1,СРЕДНЕЕ
4,0,ВЫСШЕЕ
5,2,неоконченное высшее
6,3,начальное
7,0,Высшее
8,2,НЕОКОНЧЕННОЕ ВЫСШЕЕ
9,2,Неоконченное высшее


In [31]:
family_status = data[['family_status_id', 'family_status']]

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [34]:
print(data['education'].nunique())
data['education'] = data['education'].str.lower()
print(data['education'].nunique())
print(data['family_status'].nunique())
data['family_status'] = data['family_status'].str.lower()
print(data['family_status'].nunique())

5
5
5
5


### Шаг 2.6. Категоризация дохода.

In [86]:
def total_income_function(income):
    if income <= 30000:
        return 'e'
    if 30001 <= income <= 50000:
        return 'd'
    if 50001 <= income <= 200000:
        return 'c'
    if 200001 <= income <= 1000000:
        return 'b'
    if 1000001 <= income:
        return 'a'

data['total_income_category'] = data['total_income'].apply(total_income_function)

data.groupby('total_income_category').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
total_income_category,,,,,,,,,,,,
a,25,25,25,25,25,25,25,25,25,25,25,25
b,5013,5013,5013,5013,5013,5013,5013,5013,5013,5013,5013,5013
c,15939,15939,15939,15939,15939,15939,15939,15939,15939,15939,15939,15939
d,349,349,349,349,349,349,349,349,349,349,349,349
e,22,22,22,22,22,22,22,22,22,22,22,22


### Шаг 2.7. Категоризация целей кредита.

In [87]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [88]:
def purpose_function(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жил' in purpose:
        return 'операции с недвижимостью'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'
    
data['purpose_category'] = data['purpose'].apply(purpose_function)
data['purpose_category'].unique()

array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [89]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21348 entries, 0 to 21347
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   children               21348 non-null  int64 
 1   days_employed          21348 non-null  int64 
 2   dob_years              21348 non-null  int64 
 3   education              21348 non-null  object
 4   education_id           21348 non-null  int64 
 5   family_status          21348 non-null  object
 6   family_status_id       21348 non-null  int64 
 7   gender                 21348 non-null  object
 8   income_type            21348 non-null  object
 9   debt                   21348 non-null  int64 
 10  total_income           21348 non-null  int64 
 11  purpose                21348 non-null  object
 12  total_income_category  21348 non-null  object
 13  purpose_category       21348 non-null  object
dtypes: int64(7), object(7)
memory usage: 2.3+ MB


In [90]:
data.groupby('children')['debt'].mean()

children
0    0.075353
1    0.092327
2    0.094542
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64

Можно сказать что количество детей не является определяющим фактором для возврата долга, самая высокая доля должников среди людей с 20ю детьми, самая низкая - среди бездетных. Похоже что у бездетных людей больше свободных денег, поэтому они чаще возвращают долги. Что-то произошло с категорией тех у кого 5 детей.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [91]:
data.groupby('family_status')['debt'].mean()

family_status
в разводе                0.070648
вдовец / вдова           0.066246
гражданский брак         0.092861
женат / замужем          0.075575
не женат / не замужем    0.097639
Name: debt, dtype: float64

Так же как и в случае с детьми - семейное положение не сильно влияет на процент должников. Самая высокая доля невозврата - среди холостых, самая низкая - среди вдовцов/вдов.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [92]:
data.groupby('total_income_category')['debt'].mean()

total_income_category
a    0.080000
b    0.070616
c    0.084886
d    0.060172
e    0.090909
Name: debt, dtype: float64

Самая высокая доля невозвратов среди самых бедных, тех, кто зарабатывает меньше 30000 рублей. Самые надежные должники - те, кто зарабатывает от 30 до 50 тысяч рублей. Самые бедные слои населения попадают в кредитную ловушку - они набирают кредитов, которые не могут вернуть. В то же время, неясно почему доля возврата кредитов не растет с увеличением дохода.

In [93]:
data.groupby('purpose_category')['debt'].mean()

purpose_category
операции с автомобилем      0.093436
операции с недвижимостью    0.072531
получение образования       0.092504
проведение свадьбы          0.078744
Name: debt, dtype: float64

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок??

Самая высокая доля невозвратов - среди тех, кто занимает деньги на операции с автомобилем, самая низкая - среди тех, кто занимает деньги на операции с недвижимостью. Автомобиль несет существенные расходы для автовладельца и иногда является прихотью, нежели необходимостью для владельца. Недвижимость, напротив, является инвестицией, которая приносит доход.

In [94]:
data_pivot = data.pivot_table(index=['children'], values='debt', aggfunc=['sum', 'count', 'mean'])
print(data_pivot.head(10)) 

           sum  count      mean
          debt   debt      debt
children                       
0         1063  14107  0.075353
1          444   4809  0.092327
2          194   2052  0.094542
3           27    330  0.081818
4            4     41  0.097561
5            0      9  0.000000


## Общий вывод:

Несмотря на то, что мы можем видеть, что определенные категории клиентов возвращают долги с разным успехом, общая динамика неясна - так например, частота возврата долга не растет с ростом доходов и остается примерно одинаковой среди людей с 1 - 20ю детьми.

Мы исследовали клиентские данные, такие как количество детей, семейное положение, доход и цель кредита. В результате нашего исследования, мы не нашли значительной корреляции между этими переменными и шансом возврата кредита. Для более точного скоринга можно исследовать такие переменные, относящиеся к финансовой истории клиента, как: кредитная нагрузка, размер залога и кредитная история.